## Beágyazások

Az előző példánkban nagy dimenziójú bag-of-words vektorokkal dolgoztunk, amelyek hossza `vocab_size`, és kifejezetten alacsony dimenziójú pozíciós reprezentációs vektorokból alakítottuk át őket ritka one-hot reprezentációvá. Ez a one-hot reprezentáció nem memóriahatékony, ráadásul minden szót egymástól függetlenül kezel, azaz a one-hot kódolt vektorok nem fejeznek ki semmilyen szemantikai hasonlóságot a szavak között.

Ebben az egységben továbbra is a **News AG** adathalmazt fogjuk vizsgálni. Kezdjük azzal, hogy betöltjük az adatokat, és előhívunk néhány definíciót az előző jegyzetfüzetből.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\train.csv: 29.5MB [00:01, 18.8MB/s]                            
d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\test.csv: 1.86MB [00:00, 11.2MB/s]                          


Building vocab...
Vocab size =  95812


## Mi az az embedding?

Az **embedding** ötlete az, hogy a szavakat alacsonyabb dimenziós, sűrű vektorokkal reprezentáljuk, amelyek valamilyen módon tükrözik a szó szemantikai jelentését. Később megbeszéljük, hogyan lehet értelmes szóbeágyazásokat létrehozni, de egyelőre gondoljunk az embeddingre úgy, mint egy módszerre a szóvektor dimenziójának csökkentésére.

Az embedding réteg tehát egy szót kap bemenetként, és egy meghatározott `embedding_size` méretű kimeneti vektort állít elő. Bizonyos értelemben nagyon hasonló a `Linear` réteghez, de ahelyett, hogy egy one-hot kódolt vektort venne, képes lesz egy szó számát bemenetként fogadni.

Ha az embedding réteget használjuk hálózatunk első rétegeként, akkor átállhatunk a bag-of-words modellről az **embedding bag** modellre. Ebben először minden szót a szövegünkben a megfelelő embeddingre konvertálunk, majd valamilyen aggregáló függvényt számítunk ki az összes embedding felett, például `sum`, `average` vagy `max`.

![Kép, amely egy embedding osztályozót mutat öt szekvencia szóra.](../../../../../translated_images/hu/embedding-classifier-example.b77f021a7ee67eee.webp)

Az osztályozó neurális hálózatunk embedding réteggel kezdődik, majd egy aggregáló réteggel, és végül egy lineáris osztályozóval a tetején:


In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### A változó szekvenciahossz kezelése

Ennek az architektúrának az eredményeként a hálózatunkhoz tartozó minibatch-eket egy bizonyos módon kell létrehozni. Az előző egységben, amikor bag-of-words-t (BoW) használtunk, minden BoW tenzor a minibatch-ben azonos méretű volt, `vocab_size`, függetlenül a szövegszekvencia tényleges hosszától. Amint áttérünk a szóbeágyazásokra, minden szövegmintában változó számú szó lesz, és amikor ezeket a mintákat minibatch-ekbe kombináljuk, némi kitöltést (padding) kell alkalmaznunk.

Ezt úgy érhetjük el, hogy a datasource-hoz egy `collate_fn` függvényt biztosítunk:


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### Beágyazási osztályozó tanítása

Most, hogy megfelelő adatbetöltőt definiáltunk, a korábbi egységben meghatározott tanítási függvénnyel elkezdhetjük a modell tanítását:


In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6415625
6400: acc=0.6865625
9600: acc=0.7103125
12800: acc=0.726953125
16000: acc=0.739375
19200: acc=0.75046875
22400: acc=0.7572321428571429


(0.889799795315499, 0.7623160588611644)

> **Megjegyzés**: Itt csak 25 ezer rekordot tanítunk (kevesebb, mint egy teljes epoch) az időtakarékosság érdekében, de folytathatja a tanítást, írhat egy függvényt több epoch tanítására, és kísérletezhet a tanulási ráta paraméterével a nagyobb pontosság elérése érdekében. Körülbelül 90%-os pontosságot kell tudnia elérni.


### EmbeddingBag réteg és változó hosszúságú szekvenciák reprezentációja

Az előző architektúrában minden szekvenciát ugyanarra a hosszra kellett kiegészíteni, hogy illeszkedjenek egy minibatch-be. Ez nem a leghatékonyabb módja a változó hosszúságú szekvenciák reprezentálásának – egy másik megközelítés az **offset** vektor használata, amely egy nagy vektorban tárolt összes szekvencia eltolásait tartalmazza.

![Kép, amely egy offset szekvencia reprezentációt mutat](../../../../../translated_images/hu/offset-sequence-representation.eb73fcefb29b46ee.webp)

> **Note**: A fenti képen karakterek szekvenciáját mutatjuk, de példánkban szavak szekvenciáival dolgozunk. Azonban a szekvenciák offset vektorral történő reprezentálásának általános elve ugyanaz marad.

Az offset reprezentációval való munkához az [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html) réteget használjuk. Ez hasonló az `Embedding`-hez, de tartalomvektort és offset vektort vesz bemenetként, és tartalmaz egy átlagoló réteget is, amely lehet `mean`, `sum` vagy `max`.

Íme egy módosított hálózat, amely az `EmbeddingBag`-et használja:


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

A tanulási adathalmaz előkészítéséhez egy átalakító függvényt kell biztosítanunk, amely előkészíti az eltolási vektort:


In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

Megjegyzés: az eddigi példáktól eltérően a hálózatunk most két paramétert fogad: adatvektort és eltolásvektort, amelyek különböző méretűek. Hasonlóképpen, az adatbetöltőnk is 3 értéket ad vissza 2 helyett: mind az adat-, mind az eltolásvektorokat jellemzőként biztosítja. Ezért kissé módosítanunk kell a tanítási függvényünket, hogy ezt kezelni tudjuk:


In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6153125
6400: acc=0.6615625
9600: acc=0.6932291666666667
12800: acc=0.715078125
16000: acc=0.7270625
19200: acc=0.7382291666666667
22400: acc=0.7486160714285715


(22.771553103007037, 0.7551983365323096)

## Szemantikus Beágyazások: Word2Vec

Az előző példánkban a modell beágyazási rétege megtanulta a szavakat vektoriális reprezentációvá alakítani, azonban ez a reprezentáció nem hordozott sok szemantikai jelentést. Jó lenne olyan vektoriális reprezentációt tanulni, amelyben a hasonló szavak vagy szinonimák olyan vektoroknak felelnek meg, amelyek valamilyen vektortávolság (pl. euklideszi távolság) szempontjából közel vannak egymáshoz.

Ehhez először egy nagy szöveggyűjteményen kell előre betanítanunk a beágyazási modellünket egy specifikus módon. Az egyik első módszer a szemantikus beágyazások tanítására a [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Ez két fő architektúrán alapul, amelyeket a szavak elosztott reprezentációjának előállítására használnak:

 - **Folytonos szótáska** (CBoW) — ebben az architektúrában a modellt arra tanítjuk, hogy egy szót megjósoljon a környező kontextusból. Az $(W_{-2},W_{-1},W_0,W_1,W_2)$ ngram esetén a modell célja, hogy $W_0$-t megjósolja $(W_{-2},W_{-1},W_1,W_2)$ alapján.
 - **Folytonos skip-gram** a CBoW ellentéte. A modell a környező kontextusszavak ablakát használja a jelenlegi szó megjóslására.

A CBoW gyorsabb, míg a skip-gram lassabb, de jobban reprezentálja a ritkábban előforduló szavakat.

![Kép, amely a CBoW és a Skip-Gram algoritmusokat mutatja be a szavak vektorokká alakításához.](../../../../../translated_images/hu/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Ahhoz, hogy kísérletezzünk a Google News adathalmazon előre betanított word2vec beágyazással, használhatjuk a **gensim** könyvtárat. Az alábbiakban megkeressük a 'neural' szóhoz leginkább hasonló szavakat.

> **Note:** Amikor először hozunk létre szóvektorokat, a letöltésük eltarthat egy ideig!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Számíthatunk vektorábrázolásokat is a szóból, amelyeket osztályozási modell tanításához használhatunk (a vektor első 20 komponensét mutatjuk csak a tisztánlátás érdekében):


In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

A szemantikai beágyazások nagyszerűsége abban rejlik, hogy a vektorkódolást manipulálva megváltoztathatjuk a szemantikát. Például kérhetjük, hogy találjunk egy szót, amelynek vektorképviselete a lehető legközelebb áll a *király* és *nő* szavakhoz, és a lehető legtávolabb a *férfi* szótól:


In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Mind a CBoW, mind a Skip-Gram „prediktív” beágyazások, mivel csak a helyi kontextusokat veszik figyelembe. A Word2Vec nem használja ki a globális kontextust.

A **FastText** a Word2Vec-re épít, azáltal, hogy minden szóhoz és az egyes szavakban található karakter n-gramokhoz vektorreprezentációkat tanul. Ezeket a reprezentációkat minden tanítási lépésnél átlagolják egyetlen vektorrá. Bár ez jelentősen megnöveli az előképzés számítási igényét, lehetővé teszi, hogy a szóbeágyazások kódolják az al-szó információkat.

Egy másik módszer, a **GloVe**, a társ-előfordulási mátrix ötletét használja ki, és neurális módszerekkel bontja le a társ-előfordulási mátrixot kifejezőbb és nemlineáris szóvektorokra.

Kipróbálhatod a példát azzal, hogy az embeddingeket FastText-re vagy GloVe-ra változtatod, mivel a gensim több különböző szóbeágyazási modellt is támogat.


## Előre betanított beágyazások használata PyTorch-ban

Az előző példát módosíthatjuk úgy, hogy az embedding réteg mátrixát szemantikai beágyazásokkal, például Word2Vec-kel töltsük fel. Figyelembe kell vennünk, hogy az előre betanított beágyazás és a szövegkorpusz szókészlete valószínűleg nem fog egyezni, ezért a hiányzó szavak súlyait véletlenszerű értékekkel inicializáljuk:


In [11]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.get_itos()):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


Most már képezzük a modellünket. Vegyük figyelembe, hogy a modell betanításához szükséges idő jelentősen hosszabb, mint az előző példában, mivel a beágyazási réteg mérete nagyobb, és így a paraméterek száma is sokkal magasabb. Emiatt előfordulhat, hogy több példán kell betanítanunk a modellünket, ha el akarjuk kerülni a túltanulást.


In [12]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68109375
9600: acc=0.7067708333333333
12800: acc=0.723671875
16000: acc=0.73625
19200: acc=0.7463541666666667
22400: acc=0.7560714285714286


(214.1013875559821, 0.7626759436980166)

Az esetünkben nem tapasztalunk jelentős növekedést a pontosságban, ami valószínűleg a nagyon eltérő szókészleteknek köszönhető.  
A különböző szókészletek problémájának megoldására az alábbi megoldások egyikét alkalmazhatjuk:  
* Újra betaníthatjuk a word2vec modellt a saját szókészletünkre  
* Betölthetjük az adatállományunkat a már előre betanított word2vec modell szókészletével. Az adatállomány betöltéséhez használt szókészletet a betöltés során megadhatjuk.  

Az utóbbi megközelítés egyszerűbbnek tűnik, különösen azért, mert a PyTorch `torchtext` keretrendszer beépített támogatást nyújt az embeddingekhez. Például, GloVe-alapú szókészletet a következő módon hozhatunk létre:


In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

100%|█████████▉| 399999/400000 [00:15<00:00, 25411.14it/s]


A betöltött szókincs a következő alapműveleteket tartalmazza:
* A `vocab.stoi` szótár lehetővé teszi, hogy egy szót a szótári indexévé alakítsunk.
* A `vocab.itos` az ellenkezőjét végzi - számot alakít vissza szóvá.
* A `vocab.vectors` az embedding vektorok tömbje, így egy szó `s` embeddingjének megszerzéséhez a `vocab.vectors[vocab.stoi[s]]`-t kell használnunk.

Íme egy példa az embeddingek manipulálására, amely bemutatja az egyenletet **kind-man+woman = queen** (kicsit módosítanom kellett az együtthatót, hogy működjön):


In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

Ahhoz, hogy betanítsuk az osztályozót ezekkel a beágyazásokkal, először kódolnunk kell az adatainkat a GloVe szókészlet segítségével:


In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

Amint fentebb láttuk, az összes vektorbeágyazás a `vocab.vectors` mátrixban van tárolva. Ez rendkívül egyszerűvé teszi ezeknek a súlyoknak az átmásolását az embedding réteg súlyaiba egyszerű másolással:


In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

Most edzük a modellünket, és nézzük meg, hogy jobb eredményeket kapunk-e:


In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

Az egyik oka annak, hogy nem tapasztalunk jelentős növekedést a pontosságban, az az, hogy az adatállományunkból néhány szó hiányzik az előre betanított GloVe szókészletből, és így lényegében figyelmen kívül maradnak. Ennek kiküszöbölésére saját beágyazásokat taníthatunk az adatállományunkon.


## Kontextuális Beágyazások

A hagyományos, előre betanított beágyazási reprezentációk, mint például a Word2Vec, egyik fő korlátja a szavak jelentésének egyértelműsítése. Bár az előre betanított beágyazások képesek valamennyire megragadni a szavak jelentését a kontextusban, egy szó minden lehetséges jelentését ugyanabba a beágyazásba kódolják. Ez problémákat okozhat az utófeldolgozó modellekben, mivel sok szó, például a 'play' szó, különböző jelentéssel bírhat attól függően, hogy milyen kontextusban használják.

Például a 'play' szó az alábbi két mondatban egészen eltérő jelentéssel bír:
- Elmentem egy **színdarabra** a színházba.
- John játszani szeretne a barátaival.

A fenti előre betanított beágyazások mindkét jelentést ugyanabba a beágyazásba sűrítik. Ennek a korlátnak a leküzdéséhez olyan beágyazásokat kell létrehoznunk, amelyek a **nyelvi modellen** alapulnak. Ez a modell egy nagy szövegkorpuszra van betanítva, és *tudja*, hogyan illeszkednek a szavak különböző kontextusokban. A kontextuális beágyazások részletes tárgyalása nem része ennek az oktatóanyagnak, de visszatérünk rájuk, amikor a nyelvi modellekről beszélünk a következő egységben.



---

**Felelősségkizárás**:  
Ez a dokumentum az [Co-op Translator](https://github.com/Azure/co-op-translator) AI fordítási szolgáltatás segítségével készült. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum az eredeti nyelvén tekintendő hiteles forrásnak. Kritikus információk esetén javasolt professzionális, emberi fordítást igénybe venni. Nem vállalunk felelősséget a fordítás használatából eredő félreértésekért vagy téves értelmezésekért.
